In [1]:
!pip install nltk

In [2]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download()
import pandas as pd
import matplotlib.pyplot as plt

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [3]:
fin_url = "https://finviz.com/quote.ashx?t="
tickers = ['AMZN','AAPL']

In [4]:
for ticker in tickers:
    ticker_url = fin_url + ticker
    print(ticker_url)
    req = Request(url=ticker_url, headers = {'user-agent': 'jupyter-notebook'})
    response = urlopen(req)
    
    html = bs(response, 'html.parser')
    news_table = html.find(id='news-table')
    news_tables = {ticker: news_table}
    print(news_tables)

https://finviz.com/quote.ashx?t=AMZN
{'AMZN': <table border="0" cellpadding="1" cellspacing="0" class="fullview-news-outer news-table" id="news-table" width="100%">
<tr class="cursor-pointer has-label" onclick="trackAndOpenNews(event, 'TechCrunch', 'https://finance.yahoo.com/m/328b0d50-e52e-3a6c-b84d-1541686d1566/amazon-turns-to-ai-to-help.html');">
<td align="right" width="130">
            Today 10:42AM
        </td>
<td align="left">
<div class="news-link-container">
<div class="news-link-left">
<a class="tab-link-news" href="https://finance.yahoo.com/m/328b0d50-e52e-3a6c-b84d-1541686d1566/amazon-turns-to-ai-to-help.html" rel="nofollow" target="_blank">Amazon turns to AI to help customers find clothes that fit when shopping online</a>
</div>
<div class="news-link-right">
<span>(TechCrunch)</span></div></div></td></tr>
<tr class="cursor-pointer has-label" onclick="trackAndOpenNews(event, 'Insider Monkey', 'https://finance.yahoo.com/news/17-high-growth-non-tech-145105276.html');">
<td

In [5]:
# Example for only one
amzn_data = news_tables['AAPL']
amzn_rows = amzn_data.findAll('tr')
for index, row in enumerate(amzn_rows):
    title = row.a.text
    ts = row.td.text
    
    print(ts + " " + title)
    



            Today 11:45AM
         Best Dow Jones Stocks To Buy And Watch In January 2024: Apple Rebounds, Boeing Dives

            11:39AM
         Apple Sets Date for Sales of Vision Pro Mixed-Reality Headset

            11:37AM
         Magnificent Seven Stocks To Buy And Watch: Nvidia Rallies To Record High

            11:35AM
         Evercore's Amit Daryanani on Apple: Buy the dip

            11:22AM
         Apple to launch Vision Pro headset in US on Feb. 2

            11:21AM
         These Stocks Are Moving the Most Today: Boeing, Alaska Air, Spirit AeroSystems, American, Apple, Axonics, Ambrx, and More

            10:54AM
         Apple Stock Has Been Hit. Buy the Dip, This Analyst Says.

            10:35AM
         Apple announces when the Vision Pro will go on sale

            10:30AM
         Apple Sets a Date for Sales of Vision Pro Mixed Reality Headset

            09:59AM
         Apples Vision Pro to Become Available In U.S. Feb. 2

            09:39AM
     

In [11]:
# for all

parsed_data = []
for ticker, news_table in news_tables.items():
    for row in news_table.findAll('tr'):
        title = row.a.get_text()
        ts = row.td.text.split(' ')
        
        if len(ts) == 2:
            date = ts[0]
            time = ts[1]
        else:
            time = ts[0]
            
        parsed_data.append([ticker, date, time, title])
print(parsed_data)

[['AAPL', '\r\n', '', 'Best Dow Jones Stocks To Buy And Watch In January 2024: Apple Rebounds, Boeing Dives'], ['AAPL', '\r\n', '', 'Apple Sets Date for Sales of Vision Pro Mixed-Reality Headset'], ['AAPL', '\r\n', '', 'Magnificent Seven Stocks To Buy And Watch: Nvidia Rallies To Record High'], ['AAPL', '\r\n', '', "Evercore's Amit Daryanani on Apple: Buy the dip"], ['AAPL', '\r\n', '', 'Apple to launch Vision Pro headset in US on Feb. 2'], ['AAPL', '\r\n', '', 'These Stocks Are Moving the Most Today: Boeing, Alaska Air, Spirit AeroSystems, American, Apple, Axonics, Ambrx, and More'], ['AAPL', '\r\n', '', 'Apple Stock Has Been Hit. Buy the Dip, This Analyst Says.'], ['AAPL', '\r\n', '', 'Apple announces when the Vision Pro will go on sale'], ['AAPL', '\r\n', '', 'Apple Sets a Date for Sales of Vision Pro Mixed Reality Headset'], ['AAPL', '\r\n', '', 'Apples Vision Pro to Become Available In U.S. Feb. 2'], ['AAPL', '\r\n', '', 'Amazon Has Wall Street Seeing Return to Record Stock Price'

In [12]:
df = pd.DataFrame(parsed_data, columns = ['ticker', 'date', 'time', 'title'])
df.head()

,ticker,date,time,title
0,AAPL,\r\n,,Best Dow Jones Stocks To Buy And Watch In Janu...
1,AAPL,\r\n,,Apple Sets Date for Sales of Vision Pro Mixed-...
2,AAPL,\r\n,,Magnificent Seven Stocks To Buy And Watch: Nvi...
3,AAPL,\r\n,,Evercore's Amit Daryanani on Apple: Buy the dip
4,AAPL,\r\n,,Apple to launch Vision Pro headset in US on Fe...


In [13]:
# single
vader = SIA()
print(vader.polarity_scores(" Fauzan is amazing at Stick Fight"))
print(vader.polarity_scores(" Fauzan sucks at Stick Fight"))

{'neg': 0.25, 'neu': 0.385, 'pos': 0.365, 'compound': 0.296}
{'neg': 0.63, 'neu': 0.37, 'pos': 0.0, 'compound': -0.6249}


In [14]:
# on DF

f = lambda title: vader.polarity_scores(title)['compound']
df['compound'] = df['title'].apply(f)
df.head()

,ticker,date,time,title,compound
0,AAPL,\r\n,,Best Dow Jones Stocks To Buy And Watch In Janu...,0.6369
1,AAPL,\r\n,,Apple Sets Date for Sales of Vision Pro Mixed-...,0.2500
2,AAPL,\r\n,,Magnificent Seven Stocks To Buy And Watch: Nvi...,0.5994
3,AAPL,\r\n,,Evercore's Amit Daryanani on Apple: Buy the dip,0.0000
4,AAPL,\r\n,,Apple to launch Vision Pro headset in US on Fe...,0.2500


In [15]:
df['date'] = pd.to_datetime(df.date).dt.date
plt.figure(figsize=(10,8))

mean_df = df.groupby(['ticker', 'date']).mean()
mean_df.tail()
mean_df = mean_df.unstack()
mean_df = mean_df.xs('compound', axis='columns').transpose()
mean_df.head()

DateParseError: Unable to parse datetime string: 
, at position 0

In [ ]:
mean_df.plot(kind='bar')
plt.show()


In [16]:
print(df['date'].unique())


['\r\n']
